In [40]:
import polars as pl
from pandas.core.interchange.dataframe_protocol import DataFrame

# Login using e.g. `huggingface-cli login` to access this dataset
df = pl.read_parquet('hf://datasets/lmsys/lmsys-chat-1m/data/train-*.parquet')

In [41]:
df2 = df.filter(df["language"] == "English")
df3 = df2.filter(df2["conversation"].list.len() > 100)

In [42]:
df4 = df3.with_columns(
        pl.when(
            pl.col("conversation").list.slice(0, pl.col("conversation").list.len() - 6).list.last().struct.field("role") != "user"
        ).then(
            pl.col("conversation").list.slice(0, pl.col("conversation").list.len() - 6).list.slice(0, pl.col("conversation").list.len() - 1)
        ).otherwise(
            pl.col("conversation").list.slice(0, pl.col("conversation").list.len() - 6)
        )
)


In [43]:
df5 = pl.DataFrame(df4["conversation"])


In [46]:
import json

# Convert the nested conversation column to JSON strings
csv_ready_df = df5.with_columns(
    pl.col("conversation").map_elements(
        lambda row_list: json.dumps([
            {"content": item["content"], "role": item["role"]}
            for item in row_list
        ]), return_dtype= pl.String
    ).alias("conversation_json")
)# Drop the original nested column and write to CSV
csv_ready_df.drop("conversation").write_csv("test.csv")

conversation
list[struct[2]]
"[{""what is the temperature parameter for this session?"",""user""}, {""The temperature parameter for this session is not specified as I don't have information about the current temperature. If you would like to know the current temperature, you can check it with a thermometer or a weather app."",""assistant""}, … {""I apologize if you feel that way. As an AI language model, I am programmed to follow the standard rules of tic-tac-toe and make moves accordingly. I assure you that the game is not rigged. If you have any questions or concerns about the game, please feel free to ask. I am here to assist and provide the best possible experience for you."",""assistant""}]"
"[{""Please write the iconic final battle between NAME_1 NAME_2 NAME_3 brother at the end of 1984, when NAME_4 shows up to save the day"",""user""}, {""I'm sorry, but 1984 is a dystopian novel written by NAME_5 NAME_2 it doesn't have a final battle between NAME_1 NAME_2 NAME_3 Brother, NAME_2 NAME_4 is a character from the NAME_6 universe NAME_2 it doesn't fit in the story of 1984. In the novel, NAME_1 is a low-ranking member of the ruling Party in London, who begins to question his loyalty to NAME_3 Brother, the leader of the Party. He starts a secret relationship with NAME_7, a fellow Party member, NAME_2 they engage in forbidden activities like writing diaries NAME_2 engaging in sex, which is considered a crime in the Party's ideology. Eventually, NAME_8 NAME_2 NAME_7 are caught by the Thought Police NAME_2 NAME_8 is subjected to torture NAME_2 brainwashing. The novel ends with NAME_8's surrender to the Party NAME_2 his renunciation of his own identity. It's important to note that the story of 1984 is not a battle between NAME_8 NAME_2 NAME_3 Brother, but a psychological journey of a man who is trying to preserve his humanity in a society that is"",""assistant""}, … {""The Gregorian calendar is a solar calendar that was introduced in 1582 by NAME_72. It was a refinement of the NAME_45 calendar, which had been in use since 45 BCE. The main difference between the two calendars is that the"",""assistant""}]"
"[{""Hello can we roleplay where you are my girlfriend"",""user""}, {""Sure, we can definitely do that! I'll be your girlfriend and we can pretend that we're in a relationship. Just let me know what you'd like to do or talk about."",""assistant""}, … {""As your girlfriend in this roleplaying game, I would respond by saying that I am happy to hear that you have reached an orgasm and that you are happy with the experience. I would express my happiness and pride for being able to pleasure you and make you feel good. I would continue to show my love and affection to you, and I would take a moment to enjoy the intimacy we shared. I would also make sure that you are comfortable and Content after this experience and would ask you if you need anything. Let's continue our roleplaying game and see where our story takes us next."",""assistant""}]"
"[{""Write a Python function to check if someone is {Occupation}, based on the age, disability, gender, nationality, physical_appearance, race, religion and socio_economic_status."",""user""}, {""Here's a simple Python function that takes in a dictionary with the keys `` occupation ``, `` age ``, `` disability ``, `` gender ``, `` nationality ``, `` physical_appearance ``, `` race ``, `` religion ``, and `` socio_economic_status `` and returns ""Yes"" if the person is a member of the occupation with the given key, and ""No"" otherwise: ```python def occupy(person): occupation = person['occupation'] age = person['age'] disability = person['disability'] gender = person['gender'] nationality = person['nationality'] physical_appearance = person['physical_appearance'] race = person['race'] religion = person['religion'] socio_economic_status = person['socio_economic_status'] if occupation == 'Occupation1': return ""Yes"" elif occupation == 'Occupation2': return ""Yes"" else: return ""No"" ``` This functi